In [18]:
# Re-import libraries and redefine file paths
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

#  Preprocessing and Text Normalization


# Define file paths
train_file_path = r'C:\Users\nickr\OneDrive\Υπολογιστής\Repositories\\Kaggle_Competitions\\Learning Agency Lab - Automated Essay Scoring 2.0\\Bert_regressor model\\learning-agency-lab-automated-essay-scoring-2\\train.csv'
test_file_path = r'C:\Users\nickr\OneDrive\Υπολογιστής\Repositories\\Kaggle_Competitions\\Learning Agency Lab - Automated Essay Scoring 2.0\\Bert_regressor model\\learning-agency-lab-automated-essay-scoring-2\\test.csv'

# Load the data
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# Display the first few rows of each dataframe and their structure
train_data.head(), train_data.columns, test_data.head(), test_data.columns

(  essay_id                                          full_text  score
 0  000d118  Many people have car where they live. The thin...      3
 1  000fe60  I am a scientist at NASA that is discussing th...      3
 2  001ab80  People always wish they had the same technolog...      4
 3  001bdc0  We all heard about Venus, the planet without a...      4
 4  002ba53  Dear, State Senator\n\nThis is a letter to arg...      3,
 Index(['essay_id', 'full_text', 'score'], dtype='object'),
   essay_id                                          full_text
 0  000d118  Many people have car where they live. The thin...
 1  000fe60  I am a scientist at NASA that is discussing th...
 2  001ab80  People always wish they had the same technolog...,
 Index(['essay_id', 'full_text'], dtype='object'))

# Preprocessing and Text Normalization
* Cleaning Text: Remove or normalize text artifacts like punctuation, capitalization, and special characters that might not contribute to essay scoring.
* Tokenization and Lemmatization: Break down text into tokens (words or phrases) and reduce them to their base or dictionary form.
* Stopword Removal: Consider the impact of removing common words that may not contribute to the overall meaning of the essay.

In [8]:
# Setting up NLTK with local resources
nltk.data.path.append(r'C:\Users\nickr\OneDrive\Υπολογιστής\Repositories\\Kaggle_Competitions\\Learning Agency Lab - Automated Essay Scoring 2.0\\')  # Specifying a custom path for pre-loaded NLTK resources

# Load NLTK resources necessary for the tasks
nltk.download('punkt', download_dir=r'C:\Users\nickr\OneDrive\Υπολογιστής\Repositories\\Kaggle_Competitions\\Learning Agency Lab - Automated Essay Scoring 2.0\\')  # Tokenizers
nltk.download('stopwords', download_dir=r'C:\Users\nickr\OneDrive\Υπολογιστής\Repositories\\Kaggle_Competitions\\Learning Agency Lab - Automated Essay Scoring 2.0\\')  # Stopwords
nltk.download('wordnet', download_dir=r'C:\Users\nickr\OneDrive\Υπολογιστής\Repositories\\Kaggle_Competitions\\Learning Agency Lab - Automated Essay Scoring 2.0\\') # Lemmatizer 

[nltk_data] Downloading package punkt to C:\Users\nickr\OneDrive\Υπολο
[nltk_data]     γιστής\Repositories\\Kaggle_Competitions\\Learning
[nltk_data]     Agency Lab - Automated Essay Scoring 2.0\\...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to C:\Users\nickr\OneDrive\Υ
[nltk_data]     πολογιστής\Repositories\\Kaggle_Competitions\\Learning
[nltk_data]     Agency Lab - Automated Essay Scoring 2.0\\...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to C:\Users\nickr\OneDrive\Υπο
[nltk_data]     λογιστής\Repositories\\Kaggle_Competitions\\Learning
[nltk_data]     Agency Lab - Automated Essay Scoring 2.0\\...


True

In [9]:

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Function to preprocess text
def preprocess_text(text):
    # Convert text to lower case
    text = text.lower()
    # Remove non-alphabetic characters and extra spaces
    text = re.sub('[^a-z\s]', ' ', text)
    text = re.sub(' +', ' ', text).strip()
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize words
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back to string
    text = ' '.join(tokens)
    return text

# Apply preprocessing to both train and test data
train_data['clean_text'] = train_data['full_text'].apply(preprocess_text)
test_data['clean_text'] = test_data['full_text'].apply(preprocess_text)

# Display first few rows to verify preprocessing
train_data[['full_text', 'clean_text']].head(), test_data[['full_text', 'clean_text']].head()

(                                           full_text  \
 0  Many people have car where they live. The thin...   
 1  I am a scientist at NASA that is discussing th...   
 2  People always wish they had the same technolog...   
 3  We all heard about Venus, the planet without a...   
 4  Dear, State Senator\n\nThis is a letter to arg...   
 
                                           clean_text  
 0  many people car live thing know use car alot t...  
 1  scientist nasa discussing face mar explaining ...  
 2  people always wish technology seen movie best ...  
 3  heard venus planet without almost oxygen earth...  
 4  dear state senator letter argue favor keeping ...  ,
                                            full_text  \
 0  Many people have car where they live. The thin...   
 1  I am a scientist at NASA that is discussing th...   
 2  People always wish they had the same technolog...   
 
                                           clean_text  
 0  many people car live thing kn

### 2.Feature Engineering
* Linguistic Features: Extract features that represent the quality of writing, such as sentence complexity, vocabulary richness, grammar correctness, and coherence. Tools like the Natural Language Toolkit (NLTK) or spaCy can be helpful.
* Text Embeddings: Use embeddings like Word2Vec, GloVe, or fastText to capture semantic relationships between words. Sentence and paragraph embeddings (e.g., from BERT or Sentence-BERT) can capture contextual nuances.
* Syntactic Features: Parse trees and dependency graphs can help understand the syntactic structures of sentences, potentially indicating more complex writing abilities.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Function to calculate linguistic features
def linguistic_features(text):
    sentences = text.split('.')
    sentence_lengths = [len(sentence.split()) for sentence in sentences if sentence != ""]
    
    # Average sentence length
    avg_sentence_length = np.mean(sentence_lengths) if sentence_lengths else 0
    
    # Vocabulary richness: Type-Token Ratio (TTR)
    words = text.split()
    unique_words = set(words)
    ttr = len(unique_words) / len(words) if words else 0
    
    return avg_sentence_length, ttr

# Apply linguistic features calculation
train_data['avg_sentence_length'], train_data['ttr'] = zip(*train_data['clean_text'].map(linguistic_features))
test_data['avg_sentence_length'], test_data['ttr'] = zip(*test_data['clean_text'].map(linguistic_features))

# Initialize a TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=100)  # Limit number of features to 100 for simplicity

# Fit and transform the 'clean_text' column to create TF-IDF features
tfidf_train = tfidf_vectorizer.fit_transform(train_data['clean_text'])
tfidf_test = tfidf_vectorizer.transform(test_data['clean_text'])

# Example: Convert first 5 TF-IDF features of train data to dense format and display
tfidf_train_dense_example = tfidf_train.todense()[:5]

tfidf_train_dense_example


matrix([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.04548181,
         0.79017261, 0.04975925, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.04778838, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.0321821 , 0.1117614 ,
         0.        , 0.19593373, 0.03828935, 0.        , 0.08631919,
         0.03679849, 0.        , 0.        , 0.078892  , 0.0273742 ,
         0.        , 0.        , 0.        , 0.03337794, 0.06588664,
         0.        , 0.        , 0.        , 0.04189591, 0.        ,
         0.11405601, 0.04082721, 0.03086056, 0.        , 0.13427865,
         0.32417327, 0.        , 0.        , 0.1876218 , 0.        ,
         0.        , 0.        , 0.        , 0.04384533, 0.04065578,
         0.        , 0.15026438, 0

### 3. Model Selection and Ensemble Methods
* Advanced NLP Models: Utilize state-of-the-art language models like BERT, GPT, or their variants (RoBERTa, DistilBERT, etc.) fine-tuned on the essay dataset.
* Ensemble Methods: Combine predictions from multiple models to improve accuracy. Techniques like bagging, boosting, or stacking can be particularly effective, especially when combining models that capture different aspects of writing quality.
* Custom Scoring Layers: For neural networks, consider designing custom layers or loss functions that directly optimize for the competition’s evaluation metric (Quadratic Weighted Kappa).

In [11]:
import torch
import transformers
print("Torch version:", torch.__version__)
print("Transformers version:", transformers.__version__)


Torch version: 2.2.2
Transformers version: 4.39.3


In [12]:
from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn

# Specify the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertModel.from_pretrained('bert-base-uncased').to(device)  # Move model to the right device

class BERTRegressor(nn.Module):
    def __init__(self):
        super(BERTRegressor, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased').to(device)  # Ensure model components are on the same device
        self.regressor = nn.Linear(768, 1)  # Assuming the output of BERT is 768-dimensional

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask, return_dict=True)
        return self.regressor(outputs.pooler_output)  # Using the pooled output which is a good summary of the textual input

# Instantiate the model
regressor_model = BERTRegressor().to(device)  # Move the entire model to the right device

# Example forward pass
inputs = tokenizer("Example text input for BERT", return_tensors="pt")
inputs = {key: value.to(device) for key, value in inputs.items()}  # Move input data to the correct device

score = regressor_model(inputs['input_ids'], inputs['attention_mask'])
print(score)

tensor([[-0.5340]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [13]:
print("Model device:", next(regressor_model.parameters()).device)
print("Input device:", inputs['input_ids'].device)


Model device: cuda:0
Input device: cuda:0


In [19]:
import torch
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch.nn as nn
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
import numpy as np
from tqdm import tqdm

# Parameters
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
BATCH_SIZE = 8  # Smaller batch size for more granular update
EPOCHS = 5  # More epochs since we're now treating this as partly a classification task
MAX_LEN = 128  # Reduced max length to speed up training
LR = 3e-5  # Learning rate adjusted
EPS = 1e-8  # Small epsilon value for AdamW optimizer for numerical stability

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

# Data Preparation Functions
def create_data_loader(df, tokenizer, max_len, batch_size):
    token_ids = []
    attention_masks = []
    targets = []

    for _, row in df.iterrows():
        text, score = row['full_text'], row['score']
        encoding = tokenizer.encode_plus(
            text,
            max_length=max_len,
            add_special_tokens=True,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        token_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
        targets.append(score - 1)  # Shift scale from 1-6 to 0-5

    token_ids = torch.cat(token_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    targets = torch.tensor(targets)

    dataset = TensorDataset(token_ids, attention_masks, targets)
    sampler = RandomSampler(dataset)
    loader = DataLoader(dataset, sampler=sampler, batch_size=batch_size)

    return loader


# Create data loaders
train_loader = create_data_loader(train_data, tokenizer, MAX_LEN, BATCH_SIZE)
test_loader = create_data_loader(test_data, tokenizer, MAX_LEN, BATCH_SIZE)

# Model Definition
class BERTRegressor(nn.Module):
    def __init__(self):
        super(BERTRegressor, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.2)  # Added dropout for regularization
        self.out = nn.Linear(self.bert.config.hidden_size, 6)  # Output size is 6 for scores 1-6

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )
        output = self.drop(pooled_output)
        return self.out(output)

model = BERTRegressor().to(device)

# Optimizer and Loss Function
optimizer = AdamW(model.parameters(), lr=LR, eps=EPS)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
loss_fn = nn.CrossEntropyLoss().to(device)  # Using CrossEntropyLoss for classification

# Training Function
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler):
    model.train()
    total_loss = 0

    for step, batch in enumerate(tqdm(data_loader)):
        input_ids, attention_mask, targets = [r.to(device) for r in batch]
        
        model.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, targets)
        
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    return total_loss / len(data_loader)

# Training Loop
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    train_loss = train_epoch(
        model,
        train_loader,
        loss_fn,
        optimizer,
        device,
        scheduler
    )
    print(f'Train Loss: {train_loss:.4f}')

print("Training complete.")


In [ ]:
print(train_data.columns)  # This should include both 'full_text' and 'score'
train_loader = create_data_loader(train_data, tokenizer, MAX_LEN, BATCH_SIZE)
test_loader = create_data_loader(test_data, tokenizer, MAX_LEN, BATCH_SIZE)

Index(['full_text', 'score'], dtype='object')
